In [1]:
!dir data

 驱动器 C 中的卷没有标签。
 卷的序列号是 0006-FDC5

 C:\Users\dell\dockerdata\乘用车细分市场销量预测\data 的目录

2019/09/19  14:39    <DIR>          .
2019/09/19  14:39    <DIR>          ..
2019/09/11  13:30           237,991 evaluation_public.csv
2019/09/19  14:39            51,762 mean_predicting_data.csv
2019/09/11  13:31            51,712 submit_example.csv
2019/08/23  17:36           369,997 Train.zip
2019/09/12  15:33         1,545,277 train_sales_data.csv
2019/09/12  15:33         1,399,503 train_search_data.csv
2019/09/12  15:33            47,975 train_user_reply_data.csv
               7 个文件      3,704,217 字节
               2 个目录 49,085,816,832 可用字节


In [2]:
import pandas as pd
import numpy as np

In [53]:
train_data=pd.read_csv('data/train_sales_data.csv')

In [54]:
train_data.head()

,province,adcode,model,bodyType,regYear,regMonth,salesVolume
0,上海,310000,3c974920a76ac9c1,SUV,2016,1,292
1,云南,530000,3c974920a76ac9c1,SUV,2016,1,466
2,内蒙古,150000,3c974920a76ac9c1,SUV,2016,1,257
3,北京,110000,3c974920a76ac9c1,SUV,2016,1,408
4,四川,510000,3c974920a76ac9c1,SUV,2016,1,610


In [72]:
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
train_x=train_data[['province','model','bodyType']].values
#test_x

In [73]:
model_body=train_data[['model','bodyType']].drop_duplicates()
model_body.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 1298
Data columns (total 2 columns):
model       60 non-null object
bodyType    60 non-null object
dtypes: object(2)
memory usage: 1.4+ KB


In [74]:
odec=OneHotEncoder()
train_x_encode=odec.fit_transform(train_x)

In [75]:
test_x_encode[1]

<1x90 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [76]:
x=np.concatenate((train_x_encode.toarray(),train_data[['regMonth']].values),axis=1)
y=train_data['salesVolume'].ravel()

In [77]:
x.shape,y.shape

((31680, 87), (31680,))

In [78]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.3,random_state=42)

In [79]:
from sklearn.svm import SVR
svr=SVR()
svr.fit(train_x,train_y)
svr.score(train_x,train_y)

D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


-0.06938278299641953

In [23]:
svr.score(test_x,test_y)

-0.08266312130529685

In [30]:
test_data=pd.read_csv('data/evaluation_public.csv')
test_data=pd.merge(test_data,model_body,on='model',how='left')
test_data.head(2)

,id,province,adcode,model,regYear,regMonth,forecastVolum,bodyType
0,1,上海,310000,3c974920a76ac9c1,2018,1,NaN,SUV
1,2,云南,530000,3c974920a76ac9c1,2018,1,NaN,SUV


In [34]:
test_x_cate=test_data[['province','model','bodyType']].values
test_x_encode=odec.transform(test_x_cate)
test_x=np.concatenate((test_x_encode,test_data[['regMonth']].values),axis=1)

In [40]:
test_y_pred=svr.predict(test_x)

In [50]:
test_data['forecastVolum']=test_y_pred
test_data['forecastVolum']=test_data['forecastVolum'].map(lambda x:int(x))

In [51]:
test_data.head()

,id,province,adcode,model,regYear,regMonth,forecastVolum,bodyType
0,1,上海,310000,3c974920a76ac9c1,2018,1,352,SUV
1,2,云南,530000,3c974920a76ac9c1,2018,1,353,SUV
2,3,内蒙古,150000,3c974920a76ac9c1,2018,1,358,SUV
3,4,北京,110000,3c974920a76ac9c1,2018,1,363,SUV
4,5,四川,510000,3c974920a76ac9c1,2018,1,366,SUV


In [52]:
test_data[['id','forecastVolum']].to_csv('data/svr_predicting_sales.csv',index=False)